In [26]:
# http://pytorch.org/
# Install and set up torch
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
# Use Nvidia GPU for computations if one is available, otherwise use the CPU
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'


In [27]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Set up architecture
class MyNet(nn.Module):
    # Define all layers which have learnable parameters:
    # convolutional layers, fully connected layers, batch normalization layers
    def __init__(self):
        super(MyNet, self).__init__()
        # First Convolutional Layer:
        # Input Layer is 1 since MNIST dataset has only one layer
        # Number of channels produced by the convolution is 8
        # Kernel size is set to 3
        # Padding is chosen to be 1, so one row of zeros are added 
        # on all borders of the image
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.batch1 = nn.BatchNorm2d(8) # First Batch Normalization Layer, takes in the 8 layer output of the Conv1 layer
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1) # Second Conv. Layer, reads in 8 layers from BatchNorm, produces 16 layers output
        self.batch2 = nn.BatchNorm2d(16) # Second BatchNorm layer
        self.conv3 = nn.Conv2d(16,32, kernel_size=3, padding=1) # Third Conv Layer
        self.batch3 = nn.BatchNorm2d(32) # Followed by third Batch Layer
        # Fully Connected Layer which reads in 32 layers from last batchnorm
        # Times 7*7=49 spatial pixels resulting from our 28*28px input which 
        # is then fed through a maxpool layer downsampling the image by a
        # factor of 2 --> 14*14 pixels, after BatchNorm2 there is another 
        # maxpool downsampling by a factor of 2 which leaves the image to
        # be of spatial dimensions 7*7
        # The second argument is 10 due to the number of classes being 10
        self.fc1 = nn.Linear(49*32, 10)

        # The forward pass: creating the actual architecture and order
        # of layers
    def forward(self, x):
        # Reads in x, an image or batch of images. In case of MNIST they are
        # of size of 28*28px, but pytorch does not require us to define this
        # in an input layer
        x = self.conv1(x)
        x = self.batch1(x)
        x = F.relu(x) # ReLU activation function
        x = F.max_pool2d(x,stride=2, kernel_size=2) # MaxPool to downsample x by a factor of 2
        x = self.conv2(x)
        x = self.batch2(x)
        x = F.relu(x) # ReLU activation function
        x = F.max_pool2d(x,stride=2, kernel_size=2) # Downsampling again
        x = self.conv3(x)
        x = self.batch3(x)
        x = F.relu(x) # ReLU activation function
        # Reshape x into format that fully connected layer can process -->
        # loses all spatial information; the network could be changed into
        # a segmentation task here if this fully connected layer was replaced
        # by a 1x1 convolutional layer
        x = x.view(-1, 49*32)
        x = self.fc1(x)
        # return softmax output along dimension 1 which makes sure we don't
        # take the softmax in the dimension of the batch 
        return F.log_softmax(x, dim=1)
        
# Training
# Reads in user specific arguments args, the model architecture, the train_loader
# in which the data augmentation can be achieved, the optimizer and epochs
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # Clears gradients
        output = model(data)
        loss = F.nll_loss(output, target) # Negative log likelihood loss
        loss.backward() # Perform backward pass of gradients
        optimizer.step() # Performs single optimization step
        if batch_idx % args['log_interval'] == 0:
            # Print Training Epoch, number of trained samples and loss
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
# Testing
# Reads in user specific arguments, the trained model, hardware settings and
# test loader in which no data augmentation should be performed
def test(args, model, device, test_loader):
    model.eval() # Sets the module in evaluation mode
    test_loss = 0
    correct = 0
    # Disables gradient calculation since there will not be any backward passes
    # when evaluating on the test set
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data) # Push test batch through model
            test_loss += F.nll_loss(output, target, reduction='sum').item() # Sum up batch loss
            pred = output.max(1, keepdim=True)[1]  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item() # Sum up correct matches

    test_loss /= len(test_loader.dataset)
    # Print test results for this epoch
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
100. * correct / len(test_loader.dataset)))

In [28]:
def main():
    # Training settings, user specific
    args = {
      'batch_size': 128, # default: 64
      'test_batch_size': 128, # default: 1000
      'epochs': 10, # default: 10
      'lr': 0.01, # default: 0.001 for stochastic gradient decent, but for Adam optimizer usually 0.01 is used 
      'no_cuda': False, # default: False
      'seed': 1, # default: 1
      'log_interval': 10, # default: 10
      'weight_decay': 0.0001
    }
    
    # torch.cuda.is_available() returns a bool indicating if CUDA is currently available
    use_cuda = not args['no_cuda'] and torch.cuda.is_available()

    # Sets the seed for generating random numbers for the current GPU. 
    # It is safe to call this function if CUDA is not available in which case it is ignored
    torch.manual_seed(args['seed'])
    
    device = torch.device("cuda" if use_cuda else "cpu") # Use GPU if available, else CPU

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    # Load MNIST training data
    # In general data has to be normalized by transforms.Normalize(mean, std) 
    # but obsolete for MNIST which here already comes in [0,1] doubles
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()])),
        batch_size=args['batch_size'], shuffle=True, **kwargs)
    # Load MNIST test data
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor()])),
        batch_size=args['test_batch_size'], shuffle=True, **kwargs)

    # Set the model to be MyNet
    model = MyNet().to(device)
    # Choose Adam Optimizier, the learning rate to be lr in user specific arguments, and weight decay also user set
    optimizer = optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

    # Run for as many epochs as were set in the arguments and evaluate on test set after each epoch
    for epoch in range(1, args['epochs'] + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

In [29]:
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.348931
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.566314
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.254077
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.195520
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.152640
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.123707
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.146203
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.118791
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.078516
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.124478
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.101797
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.082966
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.044225
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.121060
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.108620
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.042112
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.094222
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.066107
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.075551
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.140965
Train Epoch: 1 [

Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.042215
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.025639
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.040035
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.018431
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.033821
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.014829
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.060514
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.067764
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.026754
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.010441
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.006613
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.031701
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.133183
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.131948
Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.014718
Train Epoch: 4 [46080/60000 (77%)]	Loss: 0.003075
Train Epoch: 4 [47360/60000 (79%)]	Loss: 0.060864
Train Epoch: 4 [48640/60000 (81%)]	Loss: 0.004524
Train Epoch: 4 [49920/60000 (83%)]	Loss: 0.006694
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.035544


Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.028201
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.018077
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.033245
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.057755
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.052130

Test set: Average loss: 0.0418, Accuracy: 9863/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.024746
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.005249
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.043210
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.002695
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.006354
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.017179
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.042417
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.003724
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.011041
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.042631
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.027503
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.033281
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.039563
Train Epoch: 8 [16640/60000 (28%)]	Loss: 0.026239
Train